# Loading data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer

In [3]:
from Project_Spotify_502 import utils

In [4]:
# Load metadata and features.
tracks = utils.load('../raw_data/fma_metadata/tracks.csv')
genres = utils.load('../raw_data/fma_metadata/genres.csv')
features = utils.load('../raw_data/fma_metadata/features.csv')
echonest = utils.load('../raw_data/fma_metadata/echonest.csv')

In [47]:
main_genres = np.unique(tracks.track.genre_top[tracks['set','subset'] == "small"])

In [48]:
main_genres

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype=object)

In [45]:
def sous_genres(genre):
    our_genre_id = genres[genres.title == genre].index.to_list()[0]
    return genres[genres.parent == our_genre_id]

In [46]:
sous_genres('Rock')

,#tracks,parent,title,top_level
genre_id,,,,
25,9261,12,Punk,12
26,1952,12,Post-Rock,12
27,6041,12,Lo-Fi,12
31,1498,12,Metal,12
36,688,12,Krautrock,12
45,2469,12,Loud-Rock,12
58,2502,12,Psych-Rock,12
66,5432,12,Indie-Rock,12
70,2230,12,Industrial,12


In [ ]:
def genre_spotify_to_FMA(spotify_genre_list)

# Viz Echonest

# Exploration echonest

In [ ]:
#feature groups and nb of features in each group
echonest_feature_groups = np.unique(np.array([col[0] for col in echonest.echonest.columns])).tolist()
echonest_feature_groups_counts = np.unique(np.array([col[0] for col in echonest.echonest.columns]), return_counts=True)[1].tolist()

In [ ]:
print(echonest_feature_groups, echonest_feature_groups_counts)

In [ ]:
#plotting audio features
fig = plt.figure(figsize=(25,8))
i = 1
for feature in echonest.echonest.audio_features.columns:
        fig.add_subplot(1,len(echonest.echonest.audio_features.columns),i)
        plt.hist(echonest.echonest.audio_features[feature])
        plt.title(feature)
        i+=1

In [ ]:
# plotting ranks features
fig = plt.figure(figsize=(25,8))
i = 1
for feature in echonest.echonest.ranks.columns:
        fig.add_subplot(1,len(echonest.echonest.ranks.columns),i)
        plt.hist(echonest.echonest.ranks[feature])
        plt.title(feature)
        i+=1

In [ ]:
# plotting social features
fig = plt.figure(figsize=(25,8))
i = 1
for feature in echonest.echonest.social_features.columns:
        fig.add_subplot(1,len(echonest.echonest.social_features.columns),i)
        plt.hist(echonest.echonest.social_features[feature])
        plt.title(feature)
        i+=1

In [ ]:
#describing temporal features
echonest.echonest.temporal_features.describe()